In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, StratifiedKFold
import gensim
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("/kaggle/input/ykc-2nd/train.csv")
test = pd.read_csv("/kaggle/input/ykc-2nd/test.csv")
sub = pd.read_csv("/kaggle/input/ykc-2nd/sample_submission.csv")

In [3]:
df = pd.concat([train, test])
df = df.reset_index(drop=True)

In [4]:
target = "department_id" 
n_split = 5
features = ["product_name", "order_rate", "order_dow_mode", "order_hour_of_day_mode"]
text_features = ["product_name"]

In [5]:
train = df[~df[target].isna()]
test = df[df[target].isna()]

In [6]:
X = train.drop(target, axis=1)
y = train[target]

In [7]:
from sklearn.model_selection import train_test_split
from catboost import Pool
import sklearn.metrics
def objective(trial):
    # トレーニングデータとテストデータを分割
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
    train_pool = Pool(train_x, train_y, text_features=text_features)
    test_pool = Pool(test_x, test_y, text_features=text_features)

    # パラメータの指定
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50),
        "verbose": False
    }
    # 学習
    model = CatBoostClassifier(**params)
    model.fit(train_pool)

        ## predict on valid
    pred_val = model.predict_proba(test_x)

    ## evaluate
    score = f1_score(test_y, np.argmax(pred_val, axis = 1), average = "micro")
    print(f'F1={score}')
    return 1.0 - score

In [8]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=20)
print("Best trial:")
print(study.best_trial)

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

F1=0.7556852485409539


[I 2020-07-10 00:18:12,134] Finished trial#0 with value: 0.24431475145904613 with parameters: {'iterations': 135, 'depth': 5, 'learning_rate': 0.20059424301783854, 'random_strength': 88, 'bagging_temperature': 5.577381570223931, 'od_type': 'Iter', 'od_wait': 21}. Best is trial#0 with value: 0.24431475145904613.


F1=0.767558864962769


[I 2020-07-10 00:23:37,856] Finished trial#1 with value: 0.23244113503723096 with parameters: {'iterations': 118, 'depth': 6, 'learning_rate': 0.2674695924319836, 'random_strength': 1, 'bagging_temperature': 0.2143866582908958, 'od_type': 'IncToDec', 'od_wait': 29}. Best is trial#1 with value: 0.23244113503723096.


F1=0.4775608774401288


...

[I 2020-07-10 06:50:20,185] Finished trial#18 with value: 0.22720869390219356 with parameters: {'iterations': 299, 'depth': 8, 'learning_rate': 0.13507569370285255, 'random_strength': 68, 'bagging_temperature': 0.03489233315457894, 'od_type': 'IncToDec', 'od_wait': 43}. Best is trial#17 with value: 0.2262024552223787.


F1=0.13624471724693096


[I 2020-07-10 06:51:39,675] Finished trial#19 with value: 0.863755282753069 with parameters: {'iterations': 272, 'depth': 8, 'learning_rate': 0.12379085320090004, 'random_strength': 69, 'bagging_temperature': 88.17761525804818, 'od_type': 'IncToDec', 'od_wait': 45}. Best is trial#17 with value: 0.2262024552223787.


Best trial:
FrozenTrial(number=17, value=0.2262024552223787, datetime_start=datetime.datetime(2020, 7, 10, 5, 32, 31, 416088), datetime_complete=datetime.datetime(2020, 7, 10, 6, 11, 5, 694436), params={'iterations': 296, 'depth': 8, 'learning_rate': 0.12981336602827964, 'random_strength': 64, 'bagging_temperature': 0.19418721946609516, 'od_type': 'IncToDec', 'od_wait': 44}, distributions={'iterations': IntUniformDistribution(high=300, low=50, step=1), 'depth': IntUniformDistribution(high=10, low=4, step=1), 'learning_rate': LogUniformDistribution(high=0.3, low=0.01), 'random_strength': IntUniformDistribution(high=100, low=0, step=1), 'bagging_temperature': LogUniformDistribution(high=100.0, low=0.01), 'od_type': CategoricalDistribution(choices=('IncToDec', 'Iter')), 'od_wait': IntUniformDistribution(high=50, low=10, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE)


NameError: name 'trial' is not defined